In [1]:
{-# LANGUAGE LambdaCase #-}
{-# LANGUAGE ScopedTypeVariables #-}
{-# LANGUAGE RecordWildCards #-}

import Prelude hiding (read, replicate)
import qualified Prelude as P

import Control.Monad.ST      (ST, runST)
import Data.Array            ((!), bounds)
import Data.Graph            (Graph, buildG, vertices)
import Data.Vector.Mutable   (STVector, read, replicate, write)
import Data.STRef            (STRef, modifySTRef', newSTRef, readSTRef)
import Data.Foldable         (for_)

In [2]:
denormalise     = subtract
normalise       = (+)
other n v       = 2*n - v
clauses n [u,v] = [(other n u, v), (other n v, u)]

whenM :: Monad m => m Bool -> m () -> m ()
whenM boolM block = boolM >>= \b -> if b then block else return ()

data TState s = TState
    { index    :: STRef    s Int
    , stack    :: STRef    s [Int]
    , stackSet :: STVector s Bool
    , indices  :: STVector s (Maybe Int)
    , lowlinks :: STVector s (Maybe Int)
    , output   :: STRef    s (Maybe [[Int]])
    }

tarjan :: Int -> Graph -> Maybe [[Int]]
tarjan n graph = runST $ do
    tstate@TState{..} <- TState
        <$> newSTRef 0
        <*> newSTRef []
        <*> replicate size False
        <*> replicate size Nothing
        <*> replicate size Nothing
        <*> newSTRef (Just [])
    for_ (vertices graph) $ \v ->
        whenM ((Nothing==) <$> read indices v) $
            strongConnect n v graph tstate
    readSTRef output
    where
        size = snd (bounds graph) + 1

strongConnect :: forall s. Int -> Int -> Graph -> TState s -> ST s ()
strongConnect n v graph tstate@TState{..} = do
    i <- readSTRef index
    write indices  v (Just i)
    write lowlinks v (Just i)
    modifySTRef' index (+1)
    push v
    for_ (graph ! v) $ \w -> read indices w >>= \case
        Nothing -> do
            strongConnect n w graph tstate
            write lowlinks v =<< min <$> read lowlinks v <*> read lowlinks w
        Just{}  -> whenM (read stackSet w) $
            write lowlinks v =<< min <$> read lowlinks v <*> read indices  w
    whenM ((==) <$> read lowlinks v <*> read indices v) $
        modifySTRef' output . ((<*>) . fmap (:)) =<< addSCC n v []
    where 
        addSCC :: Int -> Int -> [Int] -> ST s (Maybe [Int])
        addSCC n v scc = do
            w <- pop
            case (other n w) `elem` scc of
                True           -> return     Nothing
                False | w == v -> return     (Just (w:scc))
                False          -> addSCC n v       (w:scc)

        push :: Int -> ST s ()
        push e = do
            modifySTRef' stack (e:)
            write stackSet e True

        pop :: ST s Int
        pop = do
            e <- head <$> readSTRef stack
            modifySTRef' stack tail
            write stackSet e False
            return e

In [3]:
checkSat :: String -> IO Bool
checkSat name = do
    p <- map (map P.read . words) . lines <$> readFile name
    let pNo    = head $ head p
        pn     = map (map (normalise pNo)) $ tail p
        pGraph = buildG (0,2*pNo) $ concatMap (clauses pNo) pn
    return . (Nothing /=) $ tarjan pNo pGraph

In [4]:
checkSat "2sat1.txt"

True

In [5]:
checkSat "2sat2.txt"

False

In [6]:
checkSat "2sat3.txt"

True

In [7]:
checkSat "2sat4.txt"

True

In [8]:
checkSat "2sat5.txt"

False

In [9]:
checkSat "2sat6.txt"

False